In [1]:
import pandas as pd
import glob
import os
import re

In [2]:
fileslist=glob.glob(r"C:\Users\Roman\Desktop\Coal_2021\*.txt")
regex_filename=r".*\\"
result = pd.DataFrame(columns=['№',
                               'Адрес отгрузки (котельная)',
                               'Дата отгрузки',
                               'Марка, гос.номер транспорта',
                               'ФИО водителя',
                               'Наименование органа исполнителя',
                               'Реквизиты транспортной накладной',
                               'Принимающий из ЖКС',
                               'Примечания',
                               'Файл'
                              ])

In [3]:
"""Поиск номера транспортной накладной и даты"""
def find_tranport_document_number(content_):
    content = content_
    start_string = "Дата"
    end_string = "Дата"
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        end_string_index = re.search(end_string,content).start()
        content = content[:end_string_index]
        return content.strip().replace(",",".")
    except:
        return ""

In [4]:
"""Поиск получателя угля"""
def find_recipient(content_):
    content = content_
    start_string_1 = "Лицо, от которого забирается груз"
    start_string_2 = "для юридического лица\)"
    end_string = "\(адрес места погрузки"
    try:
        start_string_index = re.search(start_string_1,content).end()
        content = content[start_string_index:]
        start_string_index = re.search(start_string_2,content).end()
        content = content[start_string_index:]
        end_string_index = re.search(end_string,content).start()
        content = content[:end_string_index]
        return content.strip()
    except:
        return ""

In [5]:
"""Поиск фио представителя ЖКС получателя угля"""
def find_person_recipient(content_):
    content = content_
    start_string = "Начальник котельной"
    pattern = "[А-Я][а-я]{1,20}\s[А-Я]\.[А-Я]\."
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        fio_ = re.search(pattern, content).group(0)
        return fio_.strip()
    except:
        return ""

In [6]:
"""Поиск даты и времени отгрузки угля"""
def find_datetime(content_):
    content = content_
    start_string = "\(дата и время подачи транспортного средства под выгрузку\)"
    end_string = "\(фактические дата"
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        end_string_index = re.search(end_string,content).start()
        content = content[:end_string_index]
        return content.strip()
    except:
        return ""

In [7]:
"""Поиск организации отправителя угля"""
def find_sender(content_):
    content = content_
    start_string = "Лицо, от которого забирается груз"
    end_string = "\(фамилия"
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        end_string_index = re.search(end_string,content).start()
        content = content[:end_string_index]
        return content.strip()
    except:
        return ""

In [8]:
"""Поиск автомобиля организации отправителя угля"""
def find_car_sender(content_):
    content = content_
    start_string = "Указания грузоотправителя"
    end_string = "\(параметры"
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        end_string_index = re.search(end_string,content).start()
        content = content[:end_string_index]
        return content.strip()
    except:
        return ""

In [9]:
"""Поиск фио водителя автомобиля организации отправителя угля"""
def find_person_car_sender(content_):
    content = content_
    start_string = "подписи водителя, сдавшего груз\)"
    pattern = "[А-Я][а-я]{1,20}\s[А-Я]\.[А-Я]\."
    try:
        start_string_index = re.search(start_string,content).end()
        content = content[start_string_index:]
        fio_ = re.search(pattern, content).group(0)
        return fio_.strip()
    except:
        return ""

In [10]:
data_item = {}
for i in range(0, len(fileslist)):
    filename=re.sub(regex_filename, "", fileslist[i])
    with open(fileslist[i] , "r", encoding='utf8') as f:
        content = f.read()
        data_item['№'] = i + 1
        data_item['Файл'] = filename
        data_item['Адрес отгрузки (котельная)'] = find_recipient(content)
        data_item['Дата отгрузки'] = find_datetime(content)
        data_item['Марка, гос.номер транспорта'] = find_car_sender(content)
        data_item['ФИО водителя'] = find_person_car_sender(content)
        data_item['Наименование органа исполнителя'] = find_sender(content)
        data_item['Реквизиты транспортной накладной'] = find_tranport_document_number(content)
        data_item['Принимающий из ЖКС'] = find_person_recipient(content)
        data_item['Примечания'] = ""
        result = result.append(data_item, ignore_index=True)

In [11]:
result.to_excel(r'C:\Users\Roman\Desktop\Coal_2021\result.xlsx', index=False, encoding='utf-8-sig')